### Dataframes, Spark SQL e Parquet - practica

In [1]:
# Carga o ficheiro purchases.txt desde o sistema de ficheiros localabs

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataFrame").getOrCreate()
    


In [2]:
# Crea unha táboa sobre o dataframe para poder realizar consultas SQL

In [29]:
df = spark.read.option('delimiter','\t').csv('compras')


In [3]:
# Mostra os datos do dataframe

In [30]:
df.show()

+----------+-----+--------------+--------------------+------+----------+
|       _c0|  _c1|           _c2|                 _c3|   _c4|       _c5|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [4]:
# Mostra os datos da táboa cunha consulta SQL

In [33]:
df.createOrReplaceTempView('purchases')
spark.sql('SELECT * FROM purchases LIMIT 5').show()

+----------+-----+----------+-------------------+------+----------+
|       _c0|  _c1|       _c2|                _c3|   _c4|       _c5|
+----------+-----+----------+-------------------+------+----------+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|Fort Worth|   Women's Clothing|153.57|      Visa|
|2012-01-01|09:00| San Diego|              Music| 66.08|      Cash|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|MasterCard|
+----------+-----+----------+-------------------+------+----------+



In [5]:
# Conta o número de vendas por tenda

In [38]:
spark.sql('SELECT count(_c1),_c2 from purchases group by _c2 ').show()

+----------+---------------+
|count(_c1)|            _c2|
+----------+---------------+
|     40013|North Las Vegas|
|     40333|        Phoenix|
|     40209|          Omaha|
|     39806|      Anchorage|
|     40086|        Anaheim|
|     40232|     Greensboro|
|     40368|         Dallas|
|     39728|        Oakland|
|     40342|         Laredo|
|     40173|     Scottsdale|
|     40197|    San Antonio|
|     40326|    Bakersfield|
|     40261|        Raleigh|
|     40080|    Chula Vista|
|     40748|   Philadelphia|
|     40099|     Louisville|
|     40254|    Los Angeles|
|     39826|       Chandler|
|     40561|     Sacramento|
|     40321|   Indianapolis|
+----------+---------------+
only showing top 20 rows



In [6]:
# Fai un total de vendas por categoría

In [40]:
spark.sql('SELECT sum(_c4),_c3 from purchases group by _c3 ').show()

+--------------------+--------------------+
|            sum(_c4)|                 _c3|
+--------------------+--------------------+
| 5.762482093999994E7| Children's Clothing|
| 5.759908588999996E7|      Sporting Goods|
| 5.741075303999995E7|                 CDs|
| 5.731540632000032E7|           Computers|
|5.7452374130000055E7|Consumer Electronics|
|5.7481589560001045E7|   Health and Beauty|
| 5.719725023999971E7|        Pet Supplies|
| 5.764921214000037E7|                DVDs|
|5.7491808440000996E7|                Baby|
| 5.741815449999973E7|              Crafts|
| 5.743444896999931E7|    Women's Clothing|
| 5.751316558000001E7|         Video Games|
| 5.745075790999974E7|               Books|
| 5.749548970000029E7|               Music|
| 5.762127904000029E7|      Men's Clothing|
| 5.729904664000106E7|             Cameras|
| 5.753983310999994E7|              Garden|
| 5.746347710999978E7|                Toys|
+--------------------+--------------------+



In [7]:
# Garda o dataframe en HDFS en formato parquet, con particións para as diferentes tendas

In [45]:
df.write.partitionBy("_c2").mode("overwrite").parquet("file:///tmp/compras.parquet/")


In [8]:
# Carga os datos da tenda de San Jose

In [52]:
df_parquet_partido = spark.read.parquet("file:///tmp/compras.parquet/_c2=San Jose")


In [9]:
# Mostra os datos do dataframe

In [53]:
df_parquet_partido.show()

+----------+-----+--------------------+------+----------+
|       _c0|  _c1|                 _c3|   _c4|       _c5|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [10]:
# Crea unha táboa sobre o dataframe dos datos de San Jose

In [55]:
df_parquet_partido.createOrReplaceTempView('jose')

In [11]:
# Mostra os datos da táboa

In [57]:
spark.sql('SELECT * from jose limit 10').show()

+----------+-----+--------------------+------+----------+
|       _c0|  _c1|                 _c3|   _c4|       _c5|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
+----------+-----+--------------------+------+----------+



In [12]:
# Consulta o total de vendas de San José por tipo de elementos (categoría)

In [59]:
spark.sql('SELECT SUM(_c4),_c3 from jose group by _c3').show()

+------------------+--------------------+
|          sum(_c4)|                 _c3|
+------------------+--------------------+
| 549454.2400000003| Children's Clothing|
| 540678.4100000004|      Sporting Goods|
| 586499.6699999999|                 CDs|
| 561784.6999999997|           Computers|
| 556462.0700000002|Consumer Electronics|
| 553600.1900000004|   Health and Beauty|
| 540508.1199999999|        Pet Supplies|
| 569417.1900000004|                DVDs|
|         566853.52|                Baby|
|         548947.01|              Crafts|
| 540482.1599999997|    Women's Clothing|
| 573047.9199999999|         Video Games|
|522989.79999999993|               Books|
| 550085.7599999993|               Music|
| 551148.6200000005|      Men's Clothing|
| 559343.9099999995|              Garden|
|         527568.34|             Cameras|
|         537849.78|                Toys|
+------------------+--------------------+

